# * VINSIGHT : Subscriber

In [5]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Transaction

In [6]:
''' Execute transaction '''


# Input parameter
v_start_date = 20240101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B1S000500' --Prepaid Usage Subs
        , 'TB1S000500' --Prepaid Usage Subs : TMH
        , 'DB1S000500' --Prepaid Usage Subs : DTAC
        , 'B1S000600' --Prepaid Active Caller 30D Rolling
        , 'TB1S000600' --Prepaid Active Caller 30D Rolling : TMH
        , 'DB1S000600' --Prepaid Active Caller 30D Rolling : DTAC
        , 'B1S000702' --Prepaid SubBase
        , 'DB1S000702' --Prepaid SubBase : DTAC
        , 'TB1S000702' --Prepaid SubBase : TMH
        , 'B1S000700' --Prepaid Reported SubBase
        , 'DB1S000700' --Prepaid Reported SubBase : DTAC
        , 'TB1S000700' --Prepaid Reported SubBase : TMH
        
        , 'B2S010602' --Postpaid SubBase B2C
        , 'DB2S010602' --Postpaid SubBase B2C : DTAC
        , 'TB2S010602' --Postpaid SubBase B2C : TMH
        , 'B2S010600' --Postpaid Reported SubBase B2C
        , 'DB2S010600' --Postpaid Reported SubBase B2C : DTAC
        , 'TB2S010600' --Postpaid Reported SubBase B2C : TMH
        
        , 'TB3S020604' --FTTx SubBase
        , 'TB3S000600' --FTTx Reported SubBase

        , 'TB4S000500' --TVS Active Subs
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    --AND TM_KEY_DAY IN (20240131, 20240229, 20240331, 20240430, 20240531, 20240630, 20240731, 20240831, 20240930, 20241031, 20241130, 20241231, 20250131, 20250228, 20250331) -->> Year 2024
    --AND TM_KEY_DAY IN (20250131, 20250228, 20250331, 20250430, 20250531) -->> Year 2025
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
    --ORDER BY TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\nDataFrame: {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
    
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20240101

Data as of 2025-06-06, 17:31:30

DataFrame: 8873 rows, 16 columns


In [32]:
''' Automate Currently Period '''

curr_yr = chk_src_df['TM_KEY_YR'].max()
prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_qtr = chk_src_df['TM_KEY_QTR'].max()
prev_qtr = chk_src_df['TM_KEY_QTR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = chk_src_df['TM_KEY_MTH'].drop_duplicates().sort_values().shift().max().astype(int)

curr_wk = chk_src_df['TM_KEY_WK'].max()
prev_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift().max().astype(int)
last_3_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift(3).max().astype(int)

# curr_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].max().astype(int)
# prev_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].drop_duplicates().shift().max().astype(int)

## Overview by Period

In [38]:
''' Subscriber Yearly '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'TB1S000600' #Prepaid Active Caller 30D Rolling : TMH
    , 'DB1S000600' #Prepaid Active Caller 30D Rolling : DTAC
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
    , 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    , 'TB3S000600' #FTTx Reported SubBase
	, 'TB4S000500' #TVS Active Subs
    ]

sub_yearly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_YR', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_YR
sub_yearly_df = sub_yearly_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']]
sub_yearly_df = sub_yearly_df.loc[sub_yearly_df['METRIC_CD'].isin(v_metric_list)]

sub_yearly_df['PRE_CALL_30D'] = np.where(sub_yearly_df['METRIC_CD']=='B1S000600', sub_yearly_df['P'], 0)
sub_yearly_df['PRE_CALL_30D_T'] = np.where(sub_yearly_df['METRIC_CD']=='TB1S000600', sub_yearly_df['P'], 0)
sub_yearly_df['PRE_CALL_30D_D'] = np.where(sub_yearly_df['METRIC_CD']=='DB1S000600', sub_yearly_df['P'], 0)
sub_yearly_df['POST_RPT_SUB_B2C'] = np.where(sub_yearly_df['METRIC_CD']=='B2S010600', sub_yearly_df['P'], 0)
sub_yearly_df['POST_RPT_SUB_B2C_T'] = np.where(sub_yearly_df['METRIC_CD']=='TB2S010600', sub_yearly_df['P'], 0)
sub_yearly_df['POST_RPT_SUB_B2C_D'] = np.where(sub_yearly_df['METRIC_CD']=='DB2S010600', sub_yearly_df['P'], 0)
sub_yearly_df['TOL_RPT_SUB'] = np.where(sub_yearly_df['METRIC_CD']=='TB3S000600', sub_yearly_df['P'], 0)
sub_yearly_df['TVS_ACTV_SUB'] = np.where(sub_yearly_df['METRIC_CD']=='TB4S000500', sub_yearly_df['P'], 0)

sub_yearly_df = sub_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE_CALL_30D':'sum', 'PRE_CALL_30D_T':'sum', 'PRE_CALL_30D_D':'sum', 'POST_RPT_SUB_B2C':'sum', 'POST_RPT_SUB_B2C_T':'sum', 'POST_RPT_SUB_B2C_D':'sum', 'TOL_RPT_SUB':'sum', 'TVS_ACTV_SUB':'sum'})
sub_yearly_df = sub_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
sub_yearly_df = sub_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE_CALL_30D', 'PRE_CALL_30D_T', 'PRE_CALL_30D_D', 'POST_RPT_SUB_B2C', 'POST_RPT_SUB_B2C_T', 'POST_RPT_SUB_B2C_D', 'TOL_RPT_SUB', 'TVS_ACTV_SUB']]

sub_yearly_df_display = sub_yearly_df.copy()
sub_yearly_df_display['ACTUAL_AS_OF'] = sub_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = sub_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    sub_yearly_df_display[col] = sub_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
sub_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,PRE_CALL_30D,PRE_CALL_30D_T,PRE_CALL_30D_D,POST_RPT_SUB_B2C,POST_RPT_SUB_B2C_T,POST_RPT_SUB_B2C_D,TOL_RPT_SUB,TVS_ACTV_SUB
0,2024,2025-06-06 03:26:41,20241231,"28,134,783","17,804,195","10,330,588","14,158,698","9,102,725","5,055,973","3,154,889","1,180,410"
1,2025,2025-06-06 03:26:41,20250604,"26,546,876","16,637,679","9,909,197","14,066,372","9,104,635","4,964,318","3,198,020","1,127,338"


In [40]:
''' Subscriber Quarterly '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'TB1S000600' #Prepaid Active Caller 30D Rolling : TMH
    , 'DB1S000600' #Prepaid Active Caller 30D Rolling : DTAC
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
    , 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    , 'TB3S000600' #FTTx Reported SubBase
	, 'TB4S000500' #TVS Active Subs
    ]

sub_quarterly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_QTR', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_QTR
sub_quarterly_df = sub_quarterly_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']]
sub_quarterly_df = sub_quarterly_df.loc[sub_quarterly_df['METRIC_CD'].isin(v_metric_list)]

sub_quarterly_df['PRE_CALL_30D'] = np.where(sub_quarterly_df['METRIC_CD']=='B1S000600', sub_quarterly_df['P'], 0)
sub_quarterly_df['PRE_CALL_30D_T'] = np.where(sub_quarterly_df['METRIC_CD']=='TB1S000600', sub_quarterly_df['P'], 0)
sub_quarterly_df['PRE_CALL_30D_D'] = np.where(sub_quarterly_df['METRIC_CD']=='DB1S000600', sub_quarterly_df['P'], 0)
sub_quarterly_df['POST_RPT_SUB_B2C'] = np.where(sub_quarterly_df['METRIC_CD']=='B2S010600', sub_quarterly_df['P'], 0)
sub_quarterly_df['POST_RPT_SUB_B2C_T'] = np.where(sub_quarterly_df['METRIC_CD']=='TB2S010600', sub_quarterly_df['P'], 0)
sub_quarterly_df['POST_RPT_SUB_B2C_D'] = np.where(sub_quarterly_df['METRIC_CD']=='DB2S010600', sub_quarterly_df['P'], 0)
sub_quarterly_df['TOL_RPT_SUB'] = np.where(sub_quarterly_df['METRIC_CD']=='TB3S000600', sub_quarterly_df['P'], 0)
sub_quarterly_df['TVS_ACTV_SUB'] = np.where(sub_quarterly_df['METRIC_CD']=='TB4S000500', sub_quarterly_df['P'], 0)

sub_quarterly_df = sub_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE_CALL_30D':'sum', 'PRE_CALL_30D_T':'sum', 'PRE_CALL_30D_D':'sum', 'POST_RPT_SUB_B2C':'sum', 'POST_RPT_SUB_B2C_T':'sum', 'POST_RPT_SUB_B2C_D':'sum', 'TOL_RPT_SUB':'sum', 'TVS_ACTV_SUB':'sum'})
sub_quarterly_df = sub_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
sub_quarterly_df = sub_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE_CALL_30D', 'PRE_CALL_30D_T', 'PRE_CALL_30D_D', 'POST_RPT_SUB_B2C', 'POST_RPT_SUB_B2C_T', 'POST_RPT_SUB_B2C_D', 'TOL_RPT_SUB', 'TVS_ACTV_SUB']]

sub_quarterly_df_display = sub_quarterly_df.copy()
sub_quarterly_df_display['ACTUAL_AS_OF'] = sub_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = sub_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    sub_quarterly_df_display[col] = sub_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
sub_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,PRE_CALL_30D,PRE_CALL_30D_T,PRE_CALL_30D_D,POST_RPT_SUB_B2C,POST_RPT_SUB_B2C_T,POST_RPT_SUB_B2C_D,TOL_RPT_SUB,TVS_ACTV_SUB
0,20241,2025-06-06 03:26:41,20240331,0,0,"10,558,566",0,"8,994,853",0,"3,090,287","1,258,065"
1,20242,2025-06-06 03:26:41,20240630,"27,835,150","17,348,121","10,487,029","14,226,167","9,042,184","5,183,983","3,107,084","1,236,606"
2,20243,2025-06-06 03:26:41,20240930,"27,576,181","17,261,602","10,314,579","14,186,615","9,068,470","5,118,145","3,127,304","1,205,116"
3,20244,2025-06-06 03:26:41,20241231,"28,134,783","17,804,195","10,330,588","14,158,698","9,102,725","5,055,973","3,154,889","1,180,410"
4,20251,2025-06-06 03:26:41,20250331,"27,140,596","17,215,978","9,924,618","14,168,597","9,170,605","4,997,992","3,187,717","1,153,137"
5,20252,2025-06-06 03:26:41,20250604,"26,546,876","16,637,679","9,909,197","14,066,372","9,104,635","4,964,318","3,198,020","1,127,338"


In [42]:
''' Subscriber Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'TB1S000600' #Prepaid Active Caller 30D Rolling : TMH
    , 'DB1S000600' #Prepaid Active Caller 30D Rolling : DTAC
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
    , 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    , 'TB3S000600' #FTTx Reported SubBase
	, 'TB4S000500' #TVS Active Subs
    ]

sub_monthly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_MTH', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_MTH
sub_monthly_df = sub_monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr]
sub_monthly_df = sub_monthly_df.loc[sub_monthly_df['METRIC_CD'].isin(v_metric_list)]

sub_monthly_df['PRE_CALL_30D'] = np.where(sub_monthly_df['METRIC_CD']=='B1S000600', sub_monthly_df['P'], 0)
sub_monthly_df['PRE_CALL_30D_T'] = np.where(sub_monthly_df['METRIC_CD']=='TB1S000600', sub_monthly_df['P'], 0)
sub_monthly_df['PRE_CALL_30D_D'] = np.where(sub_monthly_df['METRIC_CD']=='DB1S000600', sub_monthly_df['P'], 0)
sub_monthly_df['POST_RPT_SUB_B2C'] = np.where(sub_monthly_df['METRIC_CD']=='B2S010600', sub_monthly_df['P'], 0)
sub_monthly_df['POST_RPT_SUB_B2C_T'] = np.where(sub_monthly_df['METRIC_CD']=='TB2S010600', sub_monthly_df['P'], 0)
sub_monthly_df['POST_RPT_SUB_B2C_D'] = np.where(sub_monthly_df['METRIC_CD']=='DB2S010600', sub_monthly_df['P'], 0)
sub_monthly_df['TOL_RPT_SUB'] = np.where(sub_monthly_df['METRIC_CD']=='TB3S000600', sub_monthly_df['P'], 0)
sub_monthly_df['TVS_ACTV_SUB'] = np.where(sub_monthly_df['METRIC_CD']=='TB4S000500', sub_monthly_df['P'], 0)

sub_monthly_df = sub_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE_CALL_30D':'sum', 'PRE_CALL_30D_T':'sum', 'PRE_CALL_30D_D':'sum', 'POST_RPT_SUB_B2C':'sum', 'POST_RPT_SUB_B2C_T':'sum', 'POST_RPT_SUB_B2C_D':'sum', 'TOL_RPT_SUB':'sum', 'TVS_ACTV_SUB':'sum'})
sub_monthly_df = sub_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
sub_monthly_df = sub_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE_CALL_30D', 'PRE_CALL_30D_T', 'PRE_CALL_30D_D', 'POST_RPT_SUB_B2C', 'POST_RPT_SUB_B2C_T', 'POST_RPT_SUB_B2C_D', 'TOL_RPT_SUB', 'TVS_ACTV_SUB']]

sub_monthly_df_display = sub_monthly_df.copy()
sub_monthly_df_display['ACTUAL_AS_OF'] = sub_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = sub_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    sub_monthly_df_display[col] = sub_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
sub_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,PRE_CALL_30D,PRE_CALL_30D_T,PRE_CALL_30D_D,POST_RPT_SUB_B2C,POST_RPT_SUB_B2C_T,POST_RPT_SUB_B2C_D,TOL_RPT_SUB,TVS_ACTV_SUB
0,202401,2025-06-06 03:26:41,20240131,0,0,"10,612,127",0,0,0,0,"1,280,972"
1,202402,2025-06-06 03:26:41,20240229,0,0,"10,729,197",0,"8,998,482",0,"3,084,265","1,269,692"
2,202403,2025-06-06 03:26:41,20240331,0,0,"10,558,566",0,"8,994,853",0,"3,090,287","1,258,065"
3,202404,2025-06-06 03:26:41,20240430,0,0,"10,637,624",0,0,0,"3,091,767","1,249,008"
4,202405,2025-06-06 03:26:41,20240531,"28,067,910","17,437,100","10,630,810",0,"9,023,324",0,"3,098,514","1,245,639"
5,202406,2025-06-06 03:26:41,20240630,"27,835,150","17,348,121","10,487,029","14,226,167","9,042,184","5,183,983","3,107,084","1,236,606"
6,202407,2025-06-06 03:26:41,20240731,"27,859,766","17,397,411","10,462,355","14,209,033","9,052,569","5,156,464","3,114,835","1,226,581"
7,202408,2025-06-06 03:26:41,20240831,"27,674,309","17,257,154","10,417,155","14,190,431","9,061,002","5,129,429","3,122,029","1,214,582"
8,202409,2025-06-06 03:26:41,20240930,"27,576,181","17,261,602","10,314,579","14,186,615","9,068,470","5,118,145","3,127,304","1,205,116"
9,202410,2025-06-06 03:26:41,20241031,"27,462,871","17,233,269","10,229,602","14,179,406","9,080,143","5,099,263","3,134,367","1,195,552"


In [43]:
''' Subscriber Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'TB1S000600' #Prepaid Active Caller 30D Rolling : TMH
    , 'DB1S000600' #Prepaid Active Caller 30D Rolling : DTAC
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
    , 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    , 'TB3S000600' #FTTx Reported SubBase
	, 'TB4S000500' #TVS Active Subs
    ]

sub_weekly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_WK', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_WK
sub_weekly_df = sub_weekly_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_MTH']>=prev_mth]
sub_weekly_df = sub_weekly_df.loc[sub_weekly_df['METRIC_CD'].isin(v_metric_list)]

sub_weekly_df['PRE_CALL_30D'] = np.where(sub_weekly_df['METRIC_CD']=='B1S000600', sub_weekly_df['P'], 0)
sub_weekly_df['PRE_CALL_30D_T'] = np.where(sub_weekly_df['METRIC_CD']=='TB1S000600', sub_weekly_df['P'], 0)
sub_weekly_df['PRE_CALL_30D_D'] = np.where(sub_weekly_df['METRIC_CD']=='DB1S000600', sub_weekly_df['P'], 0)
sub_weekly_df['POST_RPT_SUB_B2C'] = np.where(sub_weekly_df['METRIC_CD']=='B2S010600', sub_weekly_df['P'], 0)
sub_weekly_df['POST_RPT_SUB_B2C_T'] = np.where(sub_weekly_df['METRIC_CD']=='TB2S010600', sub_weekly_df['P'], 0)
sub_weekly_df['POST_RPT_SUB_B2C_D'] = np.where(sub_weekly_df['METRIC_CD']=='DB2S010600', sub_weekly_df['P'], 0)
sub_weekly_df['TOL_RPT_SUB'] = np.where(sub_weekly_df['METRIC_CD']=='TB3S000600', sub_weekly_df['P'], 0)
sub_weekly_df['TVS_ACTV_SUB'] = np.where(sub_weekly_df['METRIC_CD']=='TB4S000500', sub_weekly_df['P'], 0)

sub_weekly_df = sub_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE_CALL_30D':'sum', 'PRE_CALL_30D_T':'sum', 'PRE_CALL_30D_D':'sum', 'POST_RPT_SUB_B2C':'sum', 'POST_RPT_SUB_B2C_T':'sum', 'POST_RPT_SUB_B2C_D':'sum', 'TOL_RPT_SUB':'sum', 'TVS_ACTV_SUB':'sum'})
sub_weekly_df = sub_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
sub_weekly_df = sub_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE_CALL_30D', 'PRE_CALL_30D_T', 'PRE_CALL_30D_D', 'POST_RPT_SUB_B2C', 'POST_RPT_SUB_B2C_T', 'POST_RPT_SUB_B2C_D', 'TOL_RPT_SUB', 'TVS_ACTV_SUB']]

sub_weekly_df_display = sub_weekly_df.copy()
sub_weekly_df_display['ACTUAL_AS_OF'] = sub_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = sub_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    sub_weekly_df_display[col] = sub_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
sub_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,PRE_CALL_30D,PRE_CALL_30D_T,PRE_CALL_30D_D,POST_RPT_SUB_B2C,POST_RPT_SUB_B2C_T,POST_RPT_SUB_B2C_D,TOL_RPT_SUB,TVS_ACTV_SUB
0,2025018,2025-06-06 03:26:41,20250504,"26,621,620","16,788,470","9,833,150","14,097,392","9,110,307","4,987,085","3,188,949","1,140,439"
1,2025019,2025-06-06 03:26:41,20250511,"26,564,512","16,739,957","9,824,555","14,182,297","9,200,984","4,981,313","3,192,380","1,136,865"
2,2025020,2025-06-06 03:26:41,20250518,"26,631,443","16,789,114","9,842,329","14,177,262","9,202,580","4,974,682","3,191,938","1,134,057"
3,2025021,2025-06-06 03:26:41,20250525,"26,635,512","16,747,944","9,887,568","14,068,090","9,102,460","4,965,630","3,193,370","1,131,955"
4,2025022,2025-06-06 03:26:41,20250601,"26,439,246","16,530,827","9,908,419","14,162,131","9,194,914","4,967,217","3,205,472","1,128,753"
5,2025023,2025-06-06 03:26:41,20250604,"26,546,876","16,637,679","9,909,197","14,066,372","9,104,635","4,964,318","3,198,020","1,127,338"


In [35]:
''' Subscriber Daily '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'TB1S000600' #Prepaid Active Caller 30D Rolling : TMH
    , 'DB1S000600' #Prepaid Active Caller 30D Rolling : DTAC
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
    , 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    , 'TB3S000600' #FTTx Reported SubBase
	, 'TB4S000500' #TVS Active Subs
    ]

sub_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=prev_mth].copy()
sub_daily_df = sub_daily_df.loc[sub_daily_df['METRIC_CD'].isin(v_metric_list)]

sub_daily_df['PRE_CALL_30D'] = np.where(sub_daily_df['METRIC_CD']=='B1S000600', sub_daily_df['P'], 0)
sub_daily_df['PRE_CALL_30D_T'] = np.where(sub_daily_df['METRIC_CD']=='TB1S000600', sub_daily_df['P'], 0)
sub_daily_df['PRE_CALL_30D_D'] = np.where(sub_daily_df['METRIC_CD']=='DB1S000600', sub_daily_df['P'], 0)
sub_daily_df['POST_RPT_SUB_B2C'] = np.where(sub_daily_df['METRIC_CD']=='B2S010600', sub_daily_df['P'], 0)
sub_daily_df['POST_RPT_SUB_B2C_T'] = np.where(sub_daily_df['METRIC_CD']=='TB2S010600', sub_daily_df['P'], 0)
sub_daily_df['POST_RPT_SUB_B2C_D'] = np.where(sub_daily_df['METRIC_CD']=='DB2S010600', sub_daily_df['P'], 0)
sub_daily_df['TOL_RPT_SUB'] = np.where(sub_daily_df['METRIC_CD']=='TB3S000600', sub_daily_df['P'], 0)
sub_daily_df['TVS_ACTV_SUB'] = np.where(sub_daily_df['METRIC_CD']=='TB4S000500', sub_daily_df['P'], 0)

sub_daily_df = sub_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE_CALL_30D':'sum', 'PRE_CALL_30D_T':'sum', 'PRE_CALL_30D_D':'sum', 'POST_RPT_SUB_B2C':'sum', 'POST_RPT_SUB_B2C_T':'sum', 'POST_RPT_SUB_B2C_D':'sum', 'TOL_RPT_SUB':'sum', 'TVS_ACTV_SUB':'sum'})
sub_daily_df = sub_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
sub_daily_df = sub_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE_CALL_30D', 'PRE_CALL_30D_T', 'PRE_CALL_30D_D', 'POST_RPT_SUB_B2C', 'POST_RPT_SUB_B2C_T', 'POST_RPT_SUB_B2C_D', 'TOL_RPT_SUB', 'TVS_ACTV_SUB']]

sub_daily_df_display = sub_daily_df.copy()
mod_col_list = sub_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    sub_daily_df_display[col] = sub_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
sub_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE_CALL_30D,PRE_CALL_30D_T,PRE_CALL_30D_D,POST_RPT_SUB_B2C,POST_RPT_SUB_B2C_T,POST_RPT_SUB_B2C_D,TOL_RPT_SUB,TVS_ACTV_SUB
0,202506,20250605,2025-06-06 03:26:41,0,0,0,0,0,0,0,0
1,202506,20250604,2025-06-06 03:26:41,0,0,0,0,"9,104,635",0,0,"1,127,338"
2,202506,20250603,2025-06-06 03:26:41,"26,546,876","16,637,679","9,909,197","14,066,372","9,102,054","4,964,318","3,198,020","1,127,815"
3,202506,20250602,2025-06-06 03:26:41,"26,545,204","16,643,771","9,901,433","14,071,114","9,108,731","4,962,383","3,206,516","1,128,315"
4,202506,20250601,2025-06-06 03:26:41,"26,439,246","16,530,827","9,908,419","14,162,131","9,194,914","4,967,217","3,205,472","1,128,753"
5,202505,20250531,2025-06-06 03:26:41,"26,496,116","16,589,471","9,906,645","14,160,783","9,194,703","4,966,080","3,203,127","1,129,379"
6,202505,20250530,2025-06-06 03:26:41,"26,544,725","16,645,177","9,899,548","14,065,089","9,101,218","4,963,871","3,201,195","1,129,710"
7,202505,20250529,2025-06-06 03:26:41,"26,582,772","16,700,513","9,882,259","14,159,959","9,197,701","4,962,258","3,198,852","1,130,328"
8,202505,20250528,2025-06-06 03:26:41,"26,612,350","16,722,962","9,889,388","14,167,990","9,199,904","4,968,086","3,195,276","1,130,797"
9,202505,20250527,2025-06-06 03:26:41,"26,627,728","16,730,836","9,896,892","14,166,874","9,199,895","4,966,979","3,199,020","1,130,797"


## Products Summary

### Prep Monthly Data

In [11]:
''' Monthly Summary '''

monthly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_MTH', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_MTH
monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF']).agg({'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
monthly_df['DIFF (C-P)'] = monthly_df['C'] - monthly_df['P']

monthly_df_display = monthly_df.copy()
monthly_df_display['ACTUAL_AS_OF'] = monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = monthly_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

# monthly_df_display
# monthly_df_display.loc[monthly_df_display['TM_KEY_MTH']==202501]

In [12]:
''' Parameter '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth
# v_tm_key_mth = 202505

print(f'v_tm_key_mth: {v_tm_key_mth}')

v_tm_key_mth: 202506.0


### Prepaid

In [13]:
''' Prepaid Usage Subs '''

v_metric_list = [
	'B1S000500' #Prepaid Usage Subs
    , 'TB1S000500' #Prepaid Usage Subs : TMH
    , 'DB1S000500' #Prepaid Usage Subs : DTAC
    ]

prepaid_usage_sub_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
prepaid_usage_sub_df = prepaid_usage_sub_df.loc[prepaid_usage_sub_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_usage_sub_df = prepaid_usage_sub_df.sort_values(by='METRIC_NAME').reset_index(drop=True)
prepaid_usage_sub_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)


In [14]:
''' Prepaid Active Caller 30D Rolling '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'TB1S000600' #Prepaid Active Caller 30D Rolling : TMH
    , 'DB1S000600' #Prepaid Active Caller 30D Rolling : DTAC
    ]

prepaid_call_30d_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
prepaid_call_30d_df = prepaid_call_30d_df.loc[prepaid_call_30d_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_call_30d_df = prepaid_call_30d_df.sort_values(by='METRIC_NAME').reset_index(drop=True)
prepaid_call_30d_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)


In [15]:
''' Prepaid SubBase '''

v_metric_list = [
    'B1S000702' #Prepaid SubBase
    , 'TB1S000702' #Prepaid SubBase : TMH
    , 'DB1S000702' #Prepaid SubBase : DTAC
    ]

prepaid_subbase_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
prepaid_subbase_df = prepaid_subbase_df.loc[prepaid_subbase_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_subbase_df = prepaid_subbase_df.sort_values(by='METRIC_NAME').reset_index(drop=True)
prepaid_subbase_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)


In [16]:
''' Prepaid Reported SubBase '''

v_metric_list = [
    'B1S000700' #Prepaid Reported SubBase
    , 'DB1S000700' #Prepaid Reported SubBase : DTAC
    , 'TB1S000700' #Prepaid Reported SubBase : TMH
    ]

prepaid_report_sub_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
prepaid_report_sub_df = prepaid_report_sub_df.loc[prepaid_report_sub_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_report_sub_df = prepaid_report_sub_df.sort_values(by='METRIC_NAME').reset_index(drop=True)
prepaid_report_sub_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)


### Postpaid

In [17]:
''' Postpaid SubBase B2C '''

v_metric_list = [
    'B2S010602' #Postpaid SubBase B2C
    , 'DB2S010602' #Postpaid SubBase B2C : DTAC
    , 'TB2S010602' #Postpaid SubBase B2C : TMH
    ]

postpaid_subbase_b2c_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
postpaid_subbase_b2c_df = postpaid_subbase_b2c_df.loc[postpaid_subbase_b2c_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_subbase_b2c_df = postpaid_subbase_b2c_df.sort_values(by='METRIC_NAME').reset_index(drop=True)
postpaid_subbase_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)


In [18]:
''' Postpaid Reported SubBase B2C '''

v_metric_list = [
    'B2S010600' #Postpaid Reported SubBase B2C
    , 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
    ]

postpaid_report_sub_b2c_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
postpaid_report_sub_b2c_df = postpaid_report_sub_b2c_df.loc[postpaid_report_sub_b2c_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_report_sub_b2c_df = postpaid_report_sub_b2c_df.sort_values(by='METRIC_NAME').reset_index(drop=True)
postpaid_report_sub_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)


### TOL

In [19]:
''' FTTx SubBase '''

v_metric_list = ['TB3S020604'] #FTTx SubBase

fttx_subbase_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
fttx_subbase_df = fttx_subbase_df.loc[fttx_subbase_df['TM_KEY_MTH']==v_tm_key_mth]
fttx_subbase_df = fttx_subbase_df.sort_values(by='METRIC_NAME').reset_index(drop=True)
fttx_subbase_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)


In [20]:
''' FTTx Reported SubBase '''

v_metric_list = ['TB3S000600'] #FTTx Reported SubBase

fttx_report_sub_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
fttx_report_sub_df = fttx_report_sub_df.loc[fttx_report_sub_df['TM_KEY_MTH']==v_tm_key_mth]
fttx_report_sub_df = fttx_report_sub_df.sort_values(by='METRIC_NAME').reset_index(drop=True)
fttx_report_sub_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)


### TVS

In [21]:
''' TVS Active Subs '''

v_metric_list = ['TB4S000500'] #TVS Active Subs

tvs_active_sub_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
tvs_active_sub_df = tvs_active_sub_df.loc[tvs_active_sub_df['TM_KEY_MTH']==v_tm_key_mth]
tvs_active_sub_df = tvs_active_sub_df.sort_values(by='METRIC_NAME').reset_index(drop=True)
tvs_active_sub_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)


## ** Current Issue

In [22]:
''' Issue : Prepaid Active Caller 30D Rolling : TMH (Jan-Apr 2024) '''

v_metric_cd = 'TB1S000600'

issue_prepaid_call_30d_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_prepaid_call_30d_df = issue_prepaid_call_30d_df.loc[issue_prepaid_call_30d_df['TM_KEY_MTH']>=202401]
issue_prepaid_call_30d_df = issue_prepaid_call_30d_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
issue_prepaid_call_30d_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202405,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-06-06 03:26:41,20240531,"17,437,100","17,437,100","17,414,970","17,414,970","17,414,970",0
1,202406,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-06-06 03:26:41,20240630,"17,348,121","17,348,121","17,326,088","17,326,088","17,326,088",0
2,202407,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-06-06 03:26:41,20240731,"17,397,411","17,397,411","17,373,050","17,373,050","17,373,050",0
3,202408,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-06-06 03:26:41,20240831,"17,257,154","17,257,154","17,236,889","17,236,889","17,236,889",0
4,202409,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-06-06 03:26:41,20240930,"17,261,602","17,261,602","17,243,877","17,243,877","17,243,877",0
5,202410,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-06-06 03:26:41,20241031,"17,233,269","17,233,269","17,218,887","17,218,887","17,218,887",0
6,202411,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-06-06 03:26:41,20241130,"17,288,203","17,288,203","17,270,354","17,270,354","17,270,354",0
7,202412,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-06-06 03:26:41,20241231,"17,804,195","17,804,195","17,786,014","17,786,014","17,786,014",0
8,202501,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-06-06 03:26:41,20250131,"17,344,958","17,344,958","17,330,247","17,330,167","17,330,167",0
9,202502,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-06-06 03:26:41,20250228,"17,425,203","17,425,203","17,405,731","17,405,565","17,405,565",0


In [23]:
''' Issue : Postpaid Reported SubBase B2C (Jan-May 2024) '''

v_metric_cd = 'B2S010600'

issue_postpaid_report_sub_b2c_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_postpaid_report_sub_b2c_df = issue_postpaid_report_sub_b2c_df.loc[issue_postpaid_report_sub_b2c_df['TM_KEY_MTH']>=202401]
issue_postpaid_report_sub_b2c_df = issue_postpaid_report_sub_b2c_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
issue_postpaid_report_sub_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202406,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-06-06 03:26:41,20240630,0,"14,226,167","14,212,860","14,034,154","14,212,850","-14,226,167"
1,202407,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-06-06 03:26:41,20240731,0,"14,209,033","14,195,782","14,018,425","14,195,772","-14,209,033"
2,202408,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-06-06 03:26:41,20240831,0,"14,190,431","14,177,201","13,999,593","14,177,191","-14,190,431"
3,202409,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-06-06 03:26:41,20240930,0,"14,186,615","14,173,403","13,996,349","14,173,393","-14,186,615"
4,202410,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-06-06 03:26:41,20241031,0,"14,179,406","14,166,448","13,990,677","14,166,438","-14,179,406"
5,202411,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-06-06 03:26:41,20241130,0,"14,160,522","14,154,713","13,979,080","14,154,703","-14,160,522"
6,202412,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-06-06 03:26:41,20241231,0,"14,158,698","14,152,890","13,977,330","14,152,880","-14,158,698"
7,202501,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-06-06 03:26:41,20250131,0,"14,069,220","14,063,387","13,889,946","14,063,360","-14,069,220"
8,202502,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-06-06 03:26:41,20250228,0,"14,068,955","14,063,110","14,063,094","14,063,094","-14,068,955"
9,202503,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-06-06 03:26:41,20250331,0,"14,168,597","14,162,752","14,162,740","14,162,740","-14,168,597"


In [24]:
''' Issue : Postpaid Reported SubBase B2C : TMH (Jan-May 2024) '''

v_metric_cd = 'TB2S010600'

issue_postpaid_report_sub_b2c_tmh_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_postpaid_report_sub_b2c_tmh_df = issue_postpaid_report_sub_b2c_tmh_df.loc[issue_postpaid_report_sub_b2c_tmh_df['TM_KEY_MTH']>=202401]
issue_postpaid_report_sub_b2c_tmh_df = issue_postpaid_report_sub_b2c_tmh_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
issue_postpaid_report_sub_b2c_tmh_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202402,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-06-06 03:26:41,20240229,0,"8,998,482","8,998,152","8,998,139","8,998,139","-8,998,482"
1,202403,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-06-06 03:26:41,20240331,0,"8,994,853",0,0,0,"-8,994,853"
2,202405,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-06-06 03:26:41,20240531,0,"9,023,324",0,0,0,"-9,023,324"
3,202406,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-06-06 03:26:41,20240630,0,"9,042,184","9,041,896","9,041,886","9,041,886","-9,042,184"
4,202407,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-06-06 03:26:41,20240731,0,"9,052,569","9,052,314","9,052,304","9,052,304","-9,052,569"
5,202408,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-06-06 03:26:41,20240831,0,"9,061,002","9,060,761","9,060,751","9,060,751","-9,061,002"
6,202409,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-06-06 03:26:41,20240930,0,"9,068,470","9,068,233","9,068,223","9,068,223","-9,068,470"
7,202410,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-06-06 03:26:41,20241031,0,"9,080,143","9,079,912","9,079,902","9,079,902","-9,080,143"
8,202411,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-06-06 03:26:41,20241130,0,"9,088,656","9,088,441","9,088,431","9,088,431","-9,088,656"
9,202412,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-06-06 03:26:41,20241231,0,"9,102,725","9,102,514","9,102,504","9,102,504","-9,102,725"


In [25]:
''' Issue : Postpaid Reported SubBase B2C : DTAC (Jan-May 2024) '''

v_metric_cd = 'DB2S010600'

issue_postpaid_report_sub_b2c_dtac_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_postpaid_report_sub_b2c_dtac_df = issue_postpaid_report_sub_b2c_dtac_df.loc[issue_postpaid_report_sub_b2c_dtac_df['TM_KEY_MTH']>=202401]
issue_postpaid_report_sub_b2c_dtac_df = issue_postpaid_report_sub_b2c_dtac_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
issue_postpaid_report_sub_b2c_dtac_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202406,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-06-06 03:26:41,20240630,0,"5,183,983","5,170,964","4,992,268","5,170,964","-5,183,983"
1,202407,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-06-06 03:26:41,20240731,0,"5,156,464","5,143,468","4,966,121","5,143,468","-5,156,464"
2,202408,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-06-06 03:26:41,20240831,0,"5,129,429","5,116,440","4,938,842","5,116,440","-5,129,429"
3,202409,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-06-06 03:26:41,20240930,0,"5,118,145","5,105,170","4,928,126","5,105,170","-5,118,145"
4,202410,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-06-06 03:26:41,20241031,0,"5,099,263","5,086,536","4,910,775","5,086,536","-5,099,263"
5,202411,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-06-06 03:26:41,20241130,0,"5,071,866","5,066,272","4,890,649","5,066,272","-5,071,866"
6,202412,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-06-06 03:26:41,20241231,0,"5,055,973","5,050,376","4,874,826","5,050,376","-5,055,973"
7,202501,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-06-06 03:26:41,20250131,0,"5,033,939","5,028,342","4,854,928","5,028,342","-5,033,939"
8,202502,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-06-06 03:26:41,20250228,0,"5,013,920","5,008,326","5,008,326","5,008,326","-5,013,920"
9,202503,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-06-06 03:26:41,20250331,0,"4,997,992","4,992,399","4,992,399","4,992,399","-4,997,992"
